# Map Dates

In [3]:
import pandas as pd

In [ ]:
with pd.read_json("../data/quotebank-"+str(2019)+".json.bz2", lines=True,  chunksize = 10000, compression = 'bz2') as df_reader:
    for chunk in df_reader: #we read chunk by chunk in order not to store everything in memory
        for index, row in chunk.iterrows():
            for el in row.quotations:
                if el["quoteID"] == "2019-02-10-028331":
                    print("WE ARE FUCKED")

In [49]:
def filter_dataframe(year):
# This function will be used to filter the dataset selecting the quotes of a given year containing a given word
# in a free text search fashion.

    list_df = []
    with pd.read_json("../data/quotebank-"+year+".json.bz2", lines=True,  chunksize = 10000, compression = 'bz2') as df_reader:
        import concurrent.futures
        executor = concurrent.futures.ThreadPoolExecutor(30)
        for chunk in df_reader: #we read chunk by chunk in order not to store everything in memory
            executor.submit(filter_chunk, list_df, chunk)
        executor.shutdown()
            
    df = pd.concat(list_df) #we concatenate the dataframes together to obtain a unique one
    return df

In [128]:
def filter_chunk(list_df, chunk):
    chunk.drop(columns=["articleID", "phase", "title", "url", "articleLength", "names"], inplace=True)
    chunk.quotations = chunk.quotations.transform(lambda x: filtering_func(x))
    chunk = chunk[chunk["quotations"].str.len() > 0]
    list_df.append(chunk)  #we append it to the list of dataframes
            
        
def filtering_func(lst):
    final_lst = []
    for el in lst:
        if (("Trump" in el["quotation"]) or ("Clinton" in el["quotation"])):
            final_lst.append(el["quoteID"])
    return final_lst
    

In [129]:
quoteID_date = filter_dataframe("2019")

KeyboardInterrupt: 

In [ ]:
quoteID_date

In [10]:
quoteID_date.to_pickle('2019_quoteID_date.pkl')

NameError: name 'quoteID_date' is not defined

In [4]:
quoteID_cleaned = {str(2000 + i) : pd.read_pickle('../data/' + str(2000 + i) + '_quoteID_date.pkl') for i in range(15,21)}
quoteID_cleaned

{'2015':                                                  quotations  \
 147       [2015-08-05-025201, 2015-08-05-055692, 2015-08...   
 185                  [2015-08-06-081642, 2015-08-06-061444]   
 243                  [2015-08-07-047067, 2015-08-07-046176]   
 257                                     [2015-08-07-100597]   
 667       [2015-08-19-099364, 2015-08-19-121637, 2015-08...   
 ...                                                     ...   
 10144325                                [2015-05-12-050394]   
 10144709                                [2015-07-15-021355]   
 10144886                                [2015-07-23-075404]   
 10144944                                [2015-07-27-056746]   
 10144971                                [2015-07-28-102216]   
 
                         date  
 147      2015-08-05 22:33:14  
 185      2015-08-06 17:46:00  
 243      2015-08-07 17:20:35  
 257      2015-08-07 20:46:40  
 667      2015-08-19 09:49:00  
 ...                      ... 

In [5]:
trump_cleaned = pd.read_pickle('../data/df_Trump_cleaned.pkl')
trump_cleaned

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
12,2018-07-16-000103,[ Ensuring ] the orchestrating and timing of M...,Corey Lewandowski,[Q20740735],2018-07-16 14:05:34,2,"[[Corey Lewandowski, 0.7179], [None, 0.2754], ...",[http://www.theweek.co.uk/95082/donald-trump-s...,E
66,2018-05-09-001003,300-plus years of them cold shoulders... Obama...,Charlamagne Tha God,[Q16203002],2018-05-09 11:00:00,1,"[[Charlamagne Tha God, 0.4806], [None, 0.2924]...",[https://www.portlandmercury.com/music/2018/05...,E
212,2018-08-02-002115,A politically connected contractor made a $500...,None,[],2018-08-02 06:24:59,1,"[[None, 0.7942], [President Trump, 0.1406], [K...",[https://sunlightfoundation.com/2018/08/02/tod...,E
264,2018-08-19-001084,A Wider Danger: Trump's Troubling Attacks on J...,None,[],2018-08-19 02:05:39,3,"[[None, 0.9052], [Queen Elizabeth II, 0.0948]]","[http://www.vnews.com/Forum-Aug-19-19549635, h...",E
333,2018-09-26-004202,"After the commercial, she says, `They told me ...",None,[],2018-09-26 00:00:00,2,"[[None, 0.4595], [Tom Arnold, 0.3248], [Julie ...",[http://feeds.foxnews.com/~r/foxnews/entertain...,E
...,...,...,...,...,...,...,...,...,...
5243994,2020-02-05-103219,Trump offends and disrespects the Venezuelan p...,Jorge Arreaza,[Q6623799],2020-02-05 00:00:00,11,"[[Jorge Arreaza, 0.9164], [None, 0.0726], [Pre...",[https://www.rawstory.com/2020/02/imwithfred-t...,E
5243995,2020-02-05-103235,"Trump survived, but he is the most unpopular p...",None,[],2020-02-05 23:11:42,3,"[[None, 0.8786], [Donald Trump, 0.1214]]",[https://www.wellsvilledaily.com/zz/news/20200...,E
5243996,2020-03-13-071475,"Trump tried to mitigate the issue, saying it i...",Hassan Nasrallah,[Q181182],2020-03-13 22:15:06,1,"[[Hassan Nasrallah, 0.922], [None, 0.0741], [P...",[http://israelnationalnews.com/News/News.aspx/...,E
5243997,2020-03-15-037086,Trump's do-over approach -- he unlocked $50 bi...,Newt Gingrich,[Q182788],2020-03-15 00:00:00,40,"[[Newt Gingrich, 0.5146], [None, 0.3958], [Don...",[http://uspolitics.einnews.com/article/5120893...,E


In [6]:
def filter_chunk_by_number_of_qids(initial_chunk, qid_number):
    return initial_chunk[initial_chunk["qids"].str.len() == qid_number]

In [7]:
trump_cleaned = filter_chunk_by_number_of_qids(trump_cleaned, 1)

trump_cleaned

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
12,2018-07-16-000103,[ Ensuring ] the orchestrating and timing of M...,Corey Lewandowski,[Q20740735],2018-07-16 14:05:34,2,"[[Corey Lewandowski, 0.7179], [None, 0.2754], ...",[http://www.theweek.co.uk/95082/donald-trump-s...,E
66,2018-05-09-001003,300-plus years of them cold shoulders... Obama...,Charlamagne Tha God,[Q16203002],2018-05-09 11:00:00,1,"[[Charlamagne Tha God, 0.4806], [None, 0.2924]...",[https://www.portlandmercury.com/music/2018/05...,E
366,2018-01-07-002036,All I can say is it's not a hoax. The Russians...,Lindsey Graham,[Q22212],2018-01-07 15:40:00,1,"[[Lindsey Graham, 0.5251], [None, 0.2936], [Ch...",[http://postandcourier.com/politics/would-lind...,E
1024,2018-01-16-011608,being too nonchalant about Mr. Trump's rants.,Floyd Abrams,[Q3365171],2018-01-16 20:06:13,3,"[[Floyd Abrams, 0.7512], [None, 0.2421], [Hill...",[http://www.washingtontimes.com/news/2018/jan/...,E
1091,2018-09-24-011280,Brett Kavanaugh is poised to join Neil Gorsuch...,Raul Labrador,[Q555393],2018-09-24 23:21:10,1,"[[Raul Labrador, 0.6471], [None, 0.2436], [Pre...",[http://www.spokesman.com/stories/2018/sep/25/...,E
...,...,...,...,...,...,...,...,...,...
5243971,2020-04-05-029136,To say that I'm infuriated with the recent act...,Dwight Ball,[Q5318112],2020-04-05 23:11:52,1,"[[Dwight Ball, 0.6293], [None, 0.3336], [Justi...",[https://www.cbc.ca/news/politics/trudeau-will...,E
5243994,2020-02-05-103219,Trump offends and disrespects the Venezuelan p...,Jorge Arreaza,[Q6623799],2020-02-05 00:00:00,11,"[[Jorge Arreaza, 0.9164], [None, 0.0726], [Pre...",[https://www.rawstory.com/2020/02/imwithfred-t...,E
5243996,2020-03-13-071475,"Trump tried to mitigate the issue, saying it i...",Hassan Nasrallah,[Q181182],2020-03-13 22:15:06,1,"[[Hassan Nasrallah, 0.922], [None, 0.0741], [P...",[http://israelnationalnews.com/News/News.aspx/...,E
5243997,2020-03-15-037086,Trump's do-over approach -- he unlocked $50 bi...,Newt Gingrich,[Q182788],2020-03-15 00:00:00,40,"[[Newt Gingrich, 0.5146], [None, 0.3958], [Don...",[http://uspolitics.einnews.com/article/5120893...,E


In [30]:
def years_to_try(year):
    year_lst = [str(year)]
    for i in range(2020 - year):
        year_lst.append(str(year + i + 1))
    return year_lst

In [ ]:
def update_quotes(quotes, quoteID_cleaned):
    new_quotes = quotes.copy()
    new_quotes = new_quotes.drop(columns=["date"])
    new_quotes["date"] = "no_date"
    count = 0
    for index, row in quotes.iterrows():
        if row.numOccurrences > 1:
            quoteID = row.quoteID
            earliest_date = row.date
            occurrences_lst = []
            for year in years_to_try(earliest_date.year):
                dataframe = quoteID_cleaned[year]
                occurrences = dataframe[dataframe["quotations"].transform(lambda x: quoteID in x)].date.to_list()
                occurrences_lst += occurrences
                if len(occurrences_lst) == row.numOccurrences:
                    break
            new_quotes.at[index, "date"] = occurrences_lst
        else:
            new_quotes.at[index, "date"] = [row.date]
        count += 1
        if (count % 100 == 0):
            print(count)
    return new_quotes

In [ ]:
import time

start = time.time()

merged_quotes = update_quotes(trump_cleaned[100000:190000], quoteID_cleaned)

end = time.time()
delta = end - start

print(delta)

merged_quotes

In [11]:
merged_quotes.to_pickle('../data/Trump2_with_dates.pkl')

In [24]:
merged_quotes[(merged_quotes["date"].str.len() == 0) & (merged_quotes["quoteID"] > "2019")]

,quoteID,quotation,speaker,qids,numOccurrences,probas,urls,phase,date
74210,2019-07-03-049515,More fake news bullshit trying to sow discord ...,Donald Trump Jr,[Q3713655],2,"[[Donald Trump Jr, 0.8893], [None, 0.1107]]",[http://europe.newsweek.com/trump-jr-feud-jare...,E,[]
151150,2019-10-21-015123,"Donald Trump: `John Barron' , `John Miller' , ...",Ana Navarro,[Q26923187],2,"[[Ana Navarro, 0.8606], [None, 0.1178], [Carlo...",[http://europe.newsweek.com/internet-remembers...,E,[]
205785,2019-12-23-006641,because we already have that with Donald Trump.,Stephen Colbert,[Q212886],2,"[[Stephen Colbert, 0.8917], [None, 0.082], [Ji...",[http://mobile.nytimes.com/2019/12/23/arts/tel...,E,[]
213361,2019-02-10-028331,It went from Michael Cohen interested in runni...,Michael Caputo,[Q22279201],3,"[[Michael Caputo, 0.4841], [None, 0.4012], [Mi...",[http://www.msn.com/en-us/news/politics/the-fi...,E,[]
285058,2019-09-30-044541,"It is vital that we continue to have a strong,...",Andrew Cuomo,[Q11673],3,"[[Andrew Cuomo, 0.4981], [Robert Ortt, 0.2451]...",[http://news.wbfo.org/post/rep-chris-collins-r...,E,[]
...,...,...,...,...,...,...,...,...,...
21334345,2019-09-12-007032,Anybody who thinks they're smart enough to man...,Newt Gingrich,[Q182788],2,"[[Newt Gingrich, 0.938], [None, 0.0532], [Mela...",[http://theweek.com/speedreads/864442/trump-re...,E,[]
21410739,2019-10-11-029578,I am a huge fan of the Halftime Report and nev...,Tom Steyer,[Q16189531],2,"[[Tom Steyer, 0.7344], [None, 0.2656]]",[http://feeds.foxnews.com/~r/foxnews/politics/...,E,[]
21430289,2019-02-17-003386,"Approximately 50,000 Christians are confined t...",David Curry,[Q338176],2,"[[David Curry, 0.8985], [None, 0.1016]]",[https://www.christiantoday.com/article/christ...,E,[]
21439693,2019-08-14-110006,While I applaud President Trump for putting Am...,Buddy Carter,[Q16240994],2,"[[Buddy Carter, 0.9491], [None, 0.0438], [Pres...",[http://politics.blog.ajc.com/blog/politics/th...,E,[]


In [23]:
quoteID_cleaned["2019"][quoteID_cleaned["2019"]["quotations"].transform(lambda x: "2019-02-04-014299" in x)]


,quotations,date
4820985,"[2019-02-04-014299, 2019-02-04-017176]",2019-02-04 14:15:00
11044255,"[2019-02-04-017176, 2019-02-04-014299]",2019-02-04 14:15:00
